In [5]:
import numpy as np
from gurobipy import * 

In [58]:
#useful info
# index sets: i plant, j warehouse, k retailer, l year
# 
demand = [[1000,1200,1800,1200,1000,1400,1600,1000]] 
rate = [0.2,0.2,0.25,0.2,0.2,0.25,0.25,0.2]
for t in range(2,11):
    demand_1 = []
    for j in range(len(demand[0])):
        demand_1.append(demand[0][j]+demand[0][j]*rate[j]**(t-1))
    demand.append(demand_1)

# Capacity
Capacity = [16000,12000,14000,10000,13000]
plant = [1,2,3,4,5]
warehouse = [1,2,3,4]
retail = [1,2,3,4,5,6,7,8]

# construction costs 
cstr = [[2000000,1600000,1800000,900000,1500000]]

for i in range(2,11):
    c = []
    for j in range(len(cstr[0])):
        c.append(cstr[0][j]*(1.03)**(i-1))
    cstr.append(c)

# reopen costs (plant i in year l
reopen = [[190000,150000,160000,100000,130000]]
for i in range(2,11):
    c = []
    for j in range(len(reopen[0])):
        c.append(reopen[0][j]*(1.03)**(i-1))
    reopen.append(c)
# operating costs (plant i in year l
operate = [[420000,380000,460000,280000,340000]]
for i in range(2,11):
    c = []
    for j in range(len(operate[0])):
        c.append(operate[0][j]*(1.03)**(i-1))
    operate.append(c)

# shut down costs (plant i in year l
shut_down = [[170000,120000,130000,80000,110000]]
for i in range(2,11):
    c = []
    for j in range(len(shut_down[0])):
        c.append(shut_down[0][j]*(1.03)**(i-1))
    shut_down.append(c)

# shipping costs
## plant i to Warehouse j, i=5,j=4
P_WH_2=[];P_WH_3=[];P_WH_4=[];P_WH_5=[];P_WH_6=[];P_WH_7=[];P_WH_8=[];P_WH_9=[]; P_WH_10=[]
P_WH_1 = [[120,130,80,50],
          [100,30,100,90],
          [50,70,60,30],
          [60,30,70,70],
          [60,20,40,80]]
P_to_WH = [P_WH_1,P_WH_2,P_WH_3,P_WH_4,P_WH_5,P_WH_6,P_WH_7,P_WH_8,P_WH_9,P_WH_10]
for a in range(len(P_to_WH)-1):
    for i in range(len(plant)):
        k = []
        for j in range(len(warehouse)):
            k.append(P_WH_1[i][j]*(1.03)**(a+1))
        P_to_WH[a+1].append(k)

## warehouse i to retail k
WH_R_1 = [[90,100,60,50,80,90,20,120],
           [50,70,120,40,30,90,30,80],
           [60,90,70,90,90,40,110,70],
           [70,80,90,60,100,70,60,90]]

WH_R_2=[];WH_R_3=[];WH_R_4=[];WH_R_5=[];WH_R_6=[];WH_R_7=[];WH_R_8=[];WH_R_9=[];WH_R_10=[]
WH_to_R = [WH_R_1,WH_R_2,WH_R_3,WH_R_4,WH_R_5,WH_R_6,WH_R_7,WH_R_8,WH_R_9,WH_R_10]

for a in range(len(WH_to_R)-1):
    for i in range(len(warehouse)):
        k = []
        for j in range(len(retail)):
            k.append(WH_R_1[i][j]*(1.03)**(a+1))
        WH_to_R[a+1].append(k)

In [80]:
year = list(range(1,11))
print(year)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [79]:
## setting variables
# construction binary, reopen binary, operating binary, shut down binary
# number of products produces in plant i year 
model = Model("XYZ company")

# construction costs* construction binary c + reopen costs * reopen binary r + oeprating costs * operating binary o + shut down costs * shut down binary s
# + planttoWH shipping*plant to WH binary + WHtoretail shipping* WH retail binary
# + material costs 

## binary variables
C = model.addVars(range(1,len(plant)+1), range(1,len(year)+1),vtype=GRB.CONTINUOUS,ub=1,obj = cstr, name="construction") # construction 
O = model.addVars(range(1,len(plant)+1), range(1,len(year)+1),vtype=GRB.CONTINUOUS,ub=1,obj = operate, name="Operating") # operation
R = model.addVars(range(1,len(plant)+1), range(1,len(year)+1),vtype=GRB.CONTINUOUS,ub=1,obj = reopen, name="construction") # Reopen
S = model.addVars(range(1,len(plant)+1), range(1,len(year)+1),vtype=GRB.CONTINUOUS,ub=1,obj = shut_down, name="construction") # Shut down 
# lambda
#for i in range(len(year)):
X = model.addVars(range(1,len(plant)+1), range(1,len(warehouse)+1), range(1,len(year)+1),vtype=GRB.CONTINUOUS,obj = P_to_WH, name="ship_ij") # plant to WH shipping 
Y = model.addVars(range(1,len(warehouse)+1), range(1,len(retail)+1),range(1,len(year)+1),vtype=GRB.CONTINUOUS,obj = WH_to_R, name="ship_jk") # WH to retail shipping

model.modelSense = GRB.MINIMIZE

# constraints

## binary constraints
m.addConstr() ### construction before reopen

## demand constraints

## Capacity constraints & shipping constraints

## production material constraints
### alloy

### widget

## average yearly inventory 

## both the flow into a warehouse and the flow out of a warehouse should not exceed an average of 1000 units per month.

## 

In [81]:
m = Model("test")
I = [0,1,2]
J = [3,4,5]
K = [6,7,8]
vars = m.addVars(I,J,K,vtype=GRB.BINARY,name="v")
m.update()

# print names using the vars tupledict returned by addVars
for i in I:
    for j in J:
        for k in K:
            print('%s' % (vars[i,j,k].varName))

# print names using the list returned by getVars
#for v in m.getVars():
#    print(v,'%s' % (v.varName))

v[0,3,6]
v[0,3,7]
v[0,3,8]
v[0,4,6]
v[0,4,7]
v[0,4,8]
v[0,5,6]
v[0,5,7]
v[0,5,8]
v[1,3,6]
v[1,3,7]
v[1,3,8]
v[1,4,6]
v[1,4,7]
v[1,4,8]
v[1,5,6]
v[1,5,7]
v[1,5,8]
v[2,3,6]
v[2,3,7]
v[2,3,8]
v[2,4,6]
v[2,4,7]
v[2,4,8]
v[2,5,6]
v[2,5,7]
v[2,5,8]
